In [24]:
import timit_stats as ts
import gesture as ges
import os
import matplotlib.pyplot as plt
import articulation as artic

%pylab inline

root_dir = "../USC-TIMIT/EMA/Data/M1"
t_names, m_names = zip(*ts.list_TIMIT_dir(root_dir))

gestures = {}
means = {} # key : param_name, value: dict(ges, val)
variances = {} # key : param_name, value: dict(ges, val)

articulators = ["LL", "UL", "TT", "TB", "TD", "JAW"]
domains = ["_x", "_y"]
param_names = [a+d for a in articulators for d in domains]

for i in range(len(t_names)):
    t_fname = t_names[i]
    mat_fname = m_names[i]
    gest = ts.calc_gestures(mat_fname, t_fname, filter_critical_points=True, m=0.2)
    for g in gest:
        if g not in gestures:
            gestures[g] = ges.Gesture(g)
        gestures[g].extend(gest[g])
print "Gestures calculation finished"

gestures_norm, p_max, p_min = ts.normalize_gestures(gestures)

for p in param_names:
    means[p] = {}
    variances[p] = {}

for g in gestures_norm:
    g_m = gestures_norm[g].get_mean()
    g_v = gestures_norm[g].get_variance()
    for p in param_names:
        means[p][g] = g_m[p]*(p_max[p]-p_min[p]) + p_min[p]
        variances[p][g] = g_v[p]

print "Means and variances calculated successfully"



Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


Gestures calculation finished


ValueError: max() arg is an empty sequence

In [ ]:
index = 19
trans_fname = t_names[index]
mat_fname = m_names[index]

# parse .trans file
t_starts, t_ends, phonemes, words, sentences = ts.parse_transcription(trans_fname)
# parse .mat file
params, srates = ts.parse_mat(mat_fname)
# estimate trajectory
estimated_trajectories = artic.estimate_trajectory(trans_fname, means, variances, srates["TT_x"])
# target trajectories
target_trajectories = artic.get_target_trajectories(trans_fname, means, rate=srates["TT_x"])

true_trajectories = params

In [ ]:
import copy

cmap = plt.get_cmap('gist_heat')
colors = [cmap(i) for i in range(100)]

param_names = ["LL_y", "TT_y", "TB_y", "TD_y", "TT_x", "TB_x", "TD_x"]

t_start = 1.0
t_end = 3.5
i_start = int(t_start*srates[p])
i_end = int(t_end*srates[p])

for p in param_names:
# get min and max variance and means for p
    v_min = min(variances[p].itervalues())
    v_max = max(variances[p].itervalues())
    v_range = v_max - v_min
    m_min = min(means[p].itervalues())
    m_max = max(means[p].itervalues())
# prepare figure
    fig1, ax = plt.subplots(figsize=(20, 4))
    fig1.suptitle("Parameter \"{}\"".format(p),
                  fontsize=20, fontweight='bold')
    ax.grid(color='black', linestyle='-', linewidth=1, alpha=0.3, axis='y')
    ax.set_xlabel("time, (sec)")
    ax.set_ylabel("position, (mm)")
# calc range
    ax.set_xlim(t_start, t_end)

    rate = srates[p]
    length = i_end - i_start
    y = params[p][i_start:i_end]
    p_min = min(min(y), m_min) - 0.5
    p_max = max(max(y), m_max) + 0.5

#     ax.set_ylim(p_min, p_max)
    height = p_max - p_min

    #print phonemes
    for i in range(0, len(phonemes)):
        phone = phonemes[i]
        v = variances[p][phone]
        varian = gestures_norm[phone].get_variance()
        w = exp(-50*v) # varian[p]/ sum(varian.values()))
        # w is weight of parameter for this gesture
        # w = 1 - (gestures_norm[phone].get_variance[p] v-v_min)/(v_max-v_min)
        m = means[p][phone]
        clr = colors[int(w*100)]
        if t_ends[i] > t_end or t_starts[i] < t_start:
            continue
        width = t_ends[i]-t_starts[i]
        text1 = "{0}\n{1:0.1f}".format(phonemes[i], w)
        ax.text(t_starts[i],
                p_min+height/10, text1, fontsize=10, color='black')

    num_samples = min(len(true_trajectories[p]),  len(estimated_trajectories[p]))
    
    t = range(i_start, i_end)
    t = [i / srates[p] for i in t]
    print len(t), len(true_trajectories[p]), len(estimated_trajectories[p])
    ax.plot(t, true_trajectories[p][i_start:i_end], color="black", linewidth=1)
    ax.plot(t, estimated_trajectories[p][i_start:i_end], color="blue", linewidth=1)
    ax.plot(t, target_trajectories[p][i_start:i_end], color="blue", linestyle="--", linewidth=1)


    plt.show()